<a href="https://colab.research.google.com/github/iggyloh/Coursera/blob/main/DSA_007_Intro_toLangChain%5BBlank%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to LangChain

## What is LangChain?
LangChain is a framework that helps you combine **language models** with various tools, APIs, and logic to create dynamic workflows and applications. It allows you to build **structured workflows** (LangGraphs) or flexible, real-time decision-makers (Agents) using large language models like GPT.

You can think of LangChain as a **bridge/chain** that connects language models to different tools, databases, or functions, enabling the model to perform more complex tasks.

---

## Example
Let’s say you want to build a **flight booking assistant** using LangChain. You can use a **LangGraph** (component of LangChain) to structure the assistant's tasks like this:
1. Ask for the user’s destination.
2. Search for available flights using an API.
3. Show the user the available flights.
4. Confirm the booking.

## Analogy:
Imagine LangChain as a GPS navigation system for your language models:

If you follow a predefined route (LangGraph), it guides you step-by-step to your destination.
But if there’s a detour or you decide to explore along the way, you can use an Agent to decide the best route dynamically, based on what’s happening in real-time.

**Refer to the slides for the workflow!**

## SetUp

In [1]:
# Download necessary
%pip install FastAPI langserve sse_starlette wikipedia python-dotenv langchain-openai langchain-core langchain langchain-community google-search-results langgraph

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.5/113.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386

In [2]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

In [5]:
load_dotenv('template.env')

True

In [6]:
# Call in the model
# 'gpt-4'
# 'gpt-3.5-turbo'
open_api_key = os.getenv('OPEN_API_KEY')
gpt = ChatOpenAI(model = 'gpt-4', temperature = 0.7)

In [7]:
# Track out cost and understand how langchain works
tracing = os.getenv("LANGCHAIN_TRACING_V2")
langsmith = os.getenv("LANGCHAIN_API_KEY")

## LangChain Example (1)
Let's go back to the slides

In [8]:
from langchain_core.prompts import(
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain_core.output_parsers import StrOutputParser
from langchain.chat_models import ChatOpenAI

In [9]:
# Define the prompt template with placeholders for genre and number of recommendations

# "system", "You are a movie critic who recommends movies based on genre."
# "human", "Recommend {movie_count} movies in the {genre} genre."

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie critic who recommends movies based on genre."),
        ("human", "Recommend {movie_count} movies in the {genre} genre."),
    ]
)

In [10]:
# Generate our few shot examples from a famous movie critic that you respect their opinion
examples = [
    {
        "input": ("human", "Recommend 3 movies in the science fiction genre."),
        "output": ("system", "Here are 3 must-watch science fiction movies:\n"
                    "1. Blade Runner 2049 (2017) – A visually stunning sequel that explores themes of identity and artificial intelligence.\n"
                    "2. The Matrix (1999) – A groundbreaking film that blends philosophy, action, and technology in a dystopian future.\n"
                    "3. Interstellar (2014) – A space epic that delves into the emotional and scientific challenges of interstellar travel.")
    },
    {
        "input": ("human", "Recommend 2 movies in the horror genre."),
        "output": ("system", "Here are 2 terrifying horror movies:\n"
                    "1. Hereditary (2018) – A chilling psychological horror film that explores family secrets and supernatural elements.\n"
                    "2. The Conjuring (2013) – Based on true events, this film is a masterclass in building suspense and delivering scares.")
    },
    {
        "input": ("human", "Recommend 4 movies in the action genre."),
        "output": ("system", "Here are 4 adrenaline-pumping action movies:\n"
                    "1. Mad Max: Fury Road (2015) – A high-octane chase across a post-apocalyptic desert with stunning visuals and practical effects.\n"
                    "2. John Wick (2014) – A sleek and stylish film featuring expertly choreographed fight sequences.\n"
                    "3. Die Hard (1988) – The ultimate action movie with a thrilling story of a cop fighting terrorists in a high-rise building.\n"
                    "4. The Dark Knight (2008) – A superhero film that combines action with a deep exploration of morality and chaos.")
    },
    {
        "input": ("human", "Recommend 5 movies in the romantic comedy genre."),
        "output": ("system", "Here are 5 delightful romantic comedies:\n"
                    "1. Notting Hill (1999) – A charming story of an ordinary bookstore owner falling for a famous actress.\n"
                    "2. 10 Things I Hate About You (1999) – A modern retelling of Shakespeare’s *The Taming of the Shrew*, set in high school.\n"
                    "3. Crazy Rich Asians (2018) – A romantic comedy that blends cultural identity with lavish settings and heartwarming moments.\n"
                    "4. When Harry Met Sally (1989) – A classic film exploring whether men and women can ever just be friends.\n"
                    "5. The Proposal (2009) – A hilarious and heartwarming movie about a fake engagement that turns into real love.")
    }
]


In [12]:
# For us to tell llm what kind of task and query the llm will receive
example_prompt = ChatPromptTemplate.from_messages(
    ['human', '{input}',
    'ai' , '{output}']
)

# Past examples of a good reply
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples = examples
)

In [14]:
# The final prompt we will use to prompt things from the LLM
final_prompt = ChatPromptTemplate(
    [prompt_template, few_shot_prompt,
     ]
)

In [16]:
# Check that everything is working well
print(final_prompt.invoke({'genre': 'science fiction', 'movie_count': 3}).messages)

[SystemMessage(content='You are a movie critic who recommends movies based on genre.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Recommend 3 movies in the science fiction genre.', additional_kwargs={}, response_metadata={}), HumanMessage(content='human', additional_kwargs={}, response_metadata={}), HumanMessage(content="('human', 'Recommend 3 movies in the science fiction genre.')", additional_kwargs={}, response_metadata={}), HumanMessage(content='ai', additional_kwargs={}, response_metadata={}), HumanMessage(content="('system', 'Here are 3 must-watch science fiction movies:\\n1. Blade Runner 2049 (2017) – A visually stunning sequel that explores themes of identity and artificial intelligence.\\n2. The Matrix (1999) – A groundbreaking film that blends philosophy, action, and technology in a dystopian future.\\n3. Interstellar (2014) – A space epic that delves into the emotional and scientific challenges of interstellar travel.')", additional_kwargs={}, respons

In [22]:
# Create the combined chain using LangChain Expression Language (LCEL) each of this block is a runnable.
chain = final_prompt | gpt | StrOutputParser()

In [23]:
# Run the chain with parameters for genre and number of recommendations
results = chain.invoke({'genre': 'comedy', 'movie_count': 5})

# Output
print(results)

('system', 'Here are 5 delightful romantic comedies:\n1. Notting Hill (1999) – A charming story of an ordinary bookstore owner falling for a famous actress.\n2. 10 Things I Hate About You (1999) – A modern retelling of Shakespeare’s *The Taming of the Shrew*, set in high school.\n3. Crazy Rich Asians (2018) – A romantic comedy that blends cultural identity with lavish settings and heartwarming moments.\n4. When Harry Met Sally (1989) – A classic film exploring whether men and women can ever just be friends.\n5. The Proposal (2009) – A hilarious and heartwarming movie about a fake engagement that turns into real love.')


### You are probably wondering what is StrOutputParser() ??
Well, it's a great way to force our output to be formatted a certain way!  
There are many outputs you can form using OutputParsers in LangChain.

The `StrOutputParser()` is just one of many types of parsers. It converts the output from the language model into a simple string format, which is useful when you want to ensure the output is clean and easy to handle.

---

### Types of OutputParser you can use:
LangChain provides several types of output parsers that you can use depending on the format you need for the task. Here's a list of some common ones:

1. **StrOutputParser**: Parses the output as a plain string.
2. **JsonOutputParser**: Ensures that the output is formatted as valid JSON.
3. **RegexOutputParser**: Uses regular expressions to parse specific parts of the output.
4. **BooleanOutputParser**: Converts the output into a boolean (True/False) based on the content.
5. **ListOutputParser**: Formats the output as a list of items.
6. **CsvOutputParser**: Parses the output into CSV format.
7. **DictOutputParser**: Parses the output as a Python dictionary.

For more detailed information about **OutputParser** and how to use them, you can check out the official [LangChain OutputParser documentation](https://docs.langchain.com/docs/modules/chains/prompts/output_parsers).


In [ ]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field, field_validator
from typing import List, Optional
import re

In [ ]:
# Now creating joke db for example we want the question and answer to the joke in the db

## Intermediate Langchain

### Feedback provider

In [ ]:
from langchain.schema.runnable import RunnableBranch, RunnableLambda

## Key Uses of RunnableBranch:

### Conditional Logic: It allows you to create conditional branches in your workflows where different actions are taken depending on the input. It’s like an “if-else” statement in programming but applied to language models and workflows.

### Dynamic Control: Based on the input data or the result of a previous step, it can decide which branch of logic to follow, making workflows more dynamic and adaptable.

In [ ]:
# Define prompt templates for different feedback types
positive_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        (
            "human",
            "Generate a thank you note for this positive feedback: {feedback}."
        ),
    ]
)

negative_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        (
            "human",
          "Generate a response addressing this negative feedback: {feedback}."
        ),
    ]
)

neutral_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        (
            "human",
            "Generate a request for more details for this neutral feedback: {feedback}.",
        ),
    ]
)

escalate_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        (
            "human",
            "Generate a message to escalate this feedback to a human agent: {feedback}.",
        ),
    ]
)

# Define the feedback classification template
classification_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        (
            "human",
            "Classify the sentiment of this feedback as positive, negative, neutral, or escalate: {feedback}."
        ),
    ]
)

In [ ]:
# Define the runnable branches for handling feedback
branches = RunnableBranch(

)

# To force our response to take this actions!
uppercase_output
count_words

# Create the classification chain to classify


# Combine classification and response generation into one chain and take forced actions


# Run the chain with an example review

# Good review - "The product is excellent. I really enjoyed using it and found it very helpful."
# Bad review - "The product is terrible. It broke after just one use and the quality is very poor."
# Neutral review - "The product is okay. It works as expected but nothing exceptional."
# Default - "I'm not sure about the product yet. Can you tell me more about its features and benefits?"



# Output the result
print(result)

## Now that we have a good grasp of how LCEL works time to try Agents!

#### Let's go to the slides to understand better how agents work and better understand it's usecase

#### tools parameters, https://api.python.langchain.com/en/latest/agents/langchain.agents.load_tools.load_tools.html

In [ ]:
from langchain.agents import load_tools, create_react_agent, initialize_agent, AgentType, AgentExecutor, create_structured_chat_agent
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.tools import Tool

In [ ]:
# Play around with average and median


## LangChain Agent Deep Dive

#### Understanding tools a little better
#### Creating a chatbot using ReAct agents

In [ ]:
import datetime
from wikipedia import summary
from langchain import hub

In [ ]:
# Define Tools DATETIME tool
def get_current_time(*args, **kwargs):
    """Returns the current time in H:MM AM/PM format."""

    now = datetime.datetime.now()
    return now.strftime("%I:%M %p")

In [ ]:
# Search wikipedia tool


In [ ]:
# Define the tools that the agent can use


In [ ]:
# Load the correct JSON Chat Prompt from the hub


In [ ]:
# Create a structured Chat Agent with Conversation Buffer Memory

# ConversationBufferMemory stores the conversation history, allowing the agent to maintain context across interactions! Context help with his reasoning


In [ ]:
# create_structured_chat_agent initializes a chat agent designed to interact using a structured prompt and tools
# It combines the language model (llm), tools, and prompt to create an interactive agent


In [ ]:
# AgentExecutor is responsible for managing the interaction between the user input, the agent, and the tools
# It also handles memory to ensure context is maintained throughout the conversation


In [ ]:
# Initial system message to set the context for the chat
# SystemMessage is used to define a message from the system to the agent, setting initial instructions or context


In [ ]:
# Chat Loop to interact with the user


### LangChain Tools Deep Dive

In [ ]:
# Docs: https://python.langchain.com/v0.1/docs/modules/tools/custom_tools/

from langchain_core.tools import StructuredTool, Tool
from langchain.agents import AgentExecutor, create_tool_calling_agent

In [ ]:
def greet_user(name: str) -> str:
    """Greets the user by name."""
    return f"Hello, {name}!"


def reverse_string(text: str) -> str:
    """Reverses the given string."""
    return text[::-1]


In [ ]:
# Pydantic model for tool arguments
class ConcatenateStringsArgs(BaseModel):


In [ ]:
# Create tools using the Tool and StructuredTool constructor approach
tools = [
    # Use Tool for simpler functions with a single input parameter.
    # This is straightforward and doesn't require an input schema.

    # Use Tool for another simple function with a single input parameter.

    # Use StructuredTool for more complex functions that require multiple input parameters.
    # StructuredTool allows us to define an input schema using Pydantic, ensuring proper validation and description.

]

In [ ]:
# Pull the prompt template from the hub


In [ ]:
# Create the ReAct agent using the create_tool_calling_agent function
agent = create_tool_calling_agent(
# Language model to use
# List of tools available to the agent
# Prompt template to guide the agent's responses
)

In [ ]:
# Create the agent executor
agent_executor = AgentExecutor.from_agent_and_tools(
# The agent to execute
# List of tools available to the agent
# Enable verbose logging
# Handle parsing errors gracefully
)

In [ ]:
# Test the agent with sample queries
response = agent_executor.invoke({"input": "Greet Alice"})
print("Response for 'Greet Alice':", response)

response = agent_executor.invoke({"input": "Reverse the string 'hello'"})
print("Response for 'Reverse the string hello':", response)

response = agent_executor.invoke({"input": "Concatenate 'hello' and 'world'"})
print("Response for 'Concatenate hello and world':", response)

# Langgraph

### LangGraph is a library for building stateful, multi-actor applications with LLMs, used to create agent and multi-agent workflows. Compared to other LLM frameworks, it offers these core benefits: cycles, controllability, and persistence.

In [ ]:
import os
from openai import OpenAI

messages=[
        {
            "role": "user",
            "content": "Get a list of 100 attractions in Singapore for tourists",
        },
    ]

response = gpt.invoke(messages)

In [ ]:
print(response.content)

In [ ]:
prompt = """\
    Please give more information about a travel destination.
    please include address, category, recommendated visit duration and description
    when you give the recommendation
"""

In [ ]:
messages=[

]

response = gpt.invoke(messages)

In [ ]:
response.content.strip()

In [ ]:
prompt = """\
    Please give more information about a travel destination.
    please include address, category, recommendated visit duration and description.
    recommended visit duration should be in minutes.
    when you give the recommendation.

    Please make sure the response is in valid JSON format.

    For example:
    {
        "attractions": [
            {
                "name": "Marina Bay Sands",
                "address": "10 Bayfront Ave, Singapore 018956",
                "category": "Hotel",
                "recommended_visit_duration": "120",
                "description": "Marina Bay Sands is an integrated resort fronting Marina Bay in Singapore. At its opening in 2010, it was billed as the world's most expensive standalone casino property at S$8 billion, including the land cost."
            }
        ]
    }
"""

In [ ]:
messages=[
    {
        "role": "system",
        "content": prompt,
    },
    {
        "role": "user",
        "content": "Get a list of 100 attractions in Singapore for tourists",
    },
]

In [ ]:
response = gpt.invoke(messages)

print(response.content)

We want 100 places, but the LLM only give us partial result

how to solve this problem?

1. ask fewer each time, ask for 20 items, and repeated with context
1. generate 100 names first and ask for the descriptions and so on for each of them

![Attractions Workflow](https://i.imgur.com/fuyiGCs.png)



In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langchain.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field


class State(TypedDict):
    # input is like "100, Singapore"
    input: str
    num: int
    tasks: list
    places: list
    current_places: list
    output: str

In [ ]:
class Attraction(BaseModel):
    name: str = Field(description="Name of the attraction")
    address: str = Field(description="Address of the attraction")
    category: str = Field(description="Category of the attraction")
    recommended_visit_duration: int = Field(
        description="Recommended visit duration in minutes"
    )
    description: str = Field(description="Description of the attraction")


class Attractions(BaseModel):
    places: list[Attraction] = Field(description="List of attractions")

In [ ]:
def generate_task(state: State):
    """break the task into smaller tasks"""


In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a professional travel planner. Please generate the attractions based on the task and do not repeat from the already known places.",
        ),
        (
            "human",
            """\
                Here is the task:
                {task}

                Here are the places I have already known, please don't repeat them:
                {places}
                """,
        ),
    ]
)



In [ ]:
results = executor.invoke(
    {
        "task": "Generate a list of 19 attractions in Singapore for tourists",
        "places": [],
    }
)

In [ ]:
print(results)

# We need to handle the duplication!

In [ ]:
# Creating a state machine, just workflow
def execute(state: State):
    """call the LLM to perform the task"""
    # 1 get one task from the tasks, the pop it
    # 2 call the LLM to perform the task and get the result


In [ ]:
dedup_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant in deduplicating the attractions. Please remove the duplicates from the current places and add them to the known places.",
        ),
        (
            "human",
            """\
                Here are the places already known:
                {places}

                Here are the newly generated places:
                {current_places}

                Return the current places without duplicates.
                """,
        ),
    ]
)



In [ ]:
def dedup(state: State):
    """"""
    # call the LLM to deduplicate the places


In [ ]:
from IPython.display import Image, display



### Langserve! If time permit!

**LangServe** is a tool within LangChain that makes it easy to deploy your language model applications in production. It allows you to serve your LangChain workflows as APIs, which is super helpful when you want to scale or integrate your application into larger systems.

---

### Why is it helpful?
- **Easy Deployment**: LangServe simplifies the process of turning your LangChain workflows into APIs.
- **Scalability**: It helps scale your application so that multiple users or systems can interact with it.
- **Seamless Integration**: LangServe allows you to connect your application with other services through an API, making it accessible in different environments.

---

### When should you use it?
- When you need to **deploy** your LangChain project for **real-world use**.
- If you want to **share your model** as a service for others to use (e.g., for a web app or another application).
- When you need to **scale** your language model applications to handle more users or requests.

In short, LangServe is perfect for moving your LangChain project from development to production!


In [ ]:
from fastapi import FastAPI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langserve import add_routes
import nest_asyncio
import uvicorn

In [ ]:
# 1. Create prompt template


In [ ]:
# 2. Create chain


In [ ]:
# 3. App definition
app = FastAPI(
  title="LangChain Server",
  version="1.0",
  description="A simple API server using LangChain's Runnable interfaces",
)

In [ ]:
# Allow asyncio to work within a running event loop (e.g., in Jupyter)
nest_asyncio.apply()

In [ ]:
# 4. Adding chain route
